In [98]:
p = next_prime(2^32)
q = next_prime(2^33)
N = p*q
ZM = Zmod(N)

class Authenticator(object):
    
    def __init__(self):
        self.e = 65537
        self.d = inverse_mod(self.e, lcm(p-1, q-1))
    
    def sign(self, message):
        assert(message.modulus() == N)
        return message^self.d
    
class Voter(object):
    
    def __init__(self):
        self.r = None
        
    def blind_vote(self, signer, vote):
        assert(vote.modulus() == N)
        assert(self.r is None)
        self.r = ZM.random_element()
        return vote*(self.r^signer.e)
    
    def unblind_vote(self, signed_blinded_vote):
        assert(self.r is not None)
        assert(signed_blinded_vote.modulus() == N)
        signed_vote = signed_blinded_vote*self.r^-1
        self.r = None
        return signed_vote

class Verifier(object):
    
    def verify_vote(_, signer, vote, signature):
        return vote == signature^signer.e

In [99]:
signer = Authenticator()
jim = Voter()
verifier = Verifier()

# jim first blinds his vote
jims_vote = ZM(7)
jims_blind_vote = jim.blind_vote(signer, jims_vote)

# then jim requests the signer signs jim's blind vote
# in theory, the signer could authenticate that jim
# is allowed to vote, but im keeping things simple
blind_signature = signer.sign(jims_blind_vote)

# jim then unblinds his signed and blinded vote
jims_vote_signature = jim.unblind_vote(blind_signature)

# finally some other party will verify that the signature 
# for the vote is valid under the signer's public key
assert(verifier.verify_vote(signer, jims_vote, jims_vote_signature))

# notes: poorly generated blinding values

When a Voter blinds their vote, the blinding value `r` they generate must be relatively prime to `N`. If it is not, two things happen. 

1. The voter has factored `N`, which breaks the system.

2. The blinded vote `v*r mod N` has no inverse because `gcd(r, N) != 1`. Inverting `r` is required for the correctness of the system since you need `r`'s inverse to unblind the signature.

# notes: "authenticator" vs. "verifier"

In a non-blinded voting scheme (think: what you would do in real life), there are two major parts to the protocol. The first is authN of the individual wanting to vote (i.e. You are Jim and you have proven you are Jim). The second is Jim authN'ing that he is casting a specific vote. This often is done by the same entity, which is how votes are tied to individuals.

In a blinded voting scheme, these two steps happen separately. In the code example, Jim could have proven he is Jim to the `Authenticator` during the call to `signer.sign`. Because the Authenticator is signing a blinded vote the `Authenticator` learns nothing about Jim's vote.

The `Verifier` is a separate entity that learns Jim's vote, but doesn't learn that Jim is the voter. The `Verifier` can trust the vote was legitimate because the anonymous voter provides a signature that demonstrates it had authenticated with the `Authenticator`. 